Primero creamos la clase Plane para modelar a cada avión como un objeto independiente en la simulación, con sus propios datos y comportamientos. Eso hace que el código sea más claro.

In [ ]:
import math
import random
import matplotlib.pyplot as plt
import pandas as pd

class plane:
    def __init__(self, id, minuto_aparicion):
        self.id = id 
        self.estado = "en radar"
        self.minuto_aparicion = minuto_aparicion
        self.distancia_mn_aep = 100
        self.velocidad_actual = 300
        self.tiempo_en_min_aep = self.distancia_mn_aep / (self.velocidad_actual / 60)
        self.landed_minute = None
    
    def avanzar(self, dt=1):
        
    #Avanza el avión dt minutos hacia AEP, según su velocidad actual.
    
        if self.estado == "en radar" or self.estado == "volando":
            mn_por_min = self.velocidad_actual / 60
            self.distancia_mn_aep = max(0.0, self.distancia_mn_aep - mn_por_min * dt)
            if self.distancia_mn_aep == 0:
                self.estado = "aterrizó"
        elif self.estado == "desviado":
            mn_por_min = 200 / 60
            self.distancia_mn_aep += mn_por_min * dt
            if self.distancia_mn_aep >= 100:
                self.estado = "montevideo"

    def cambiar_velocidad(self, nueva_velocidad):
        # COMPLETAR
        pass
  

Se inicia cada avión cuando entra al radar (por eso los valores default)
El ID es para identificar los distintos aviones
El estado para saber la situación del avión: en radar, volando, esperando, aterrizando, desviado, montevideo (chequear)
Se resgitra en que minuto de las 18 horas apareció
La distancia en millas náuticas al AEP (en principio 100mn)
La velocidad actual en nudos (a máxima velocidad va a 300 nudos): Sirve para:
- Calcular cuánto avanza cada minuto (`distancia -= velocidad/60`).
- Aplicar reglas de congestión (reducir 20 nudos, respetar mínimos y máximos por tramo).
- Simular desvíos a 200 nudos.
El tiempo que le falta para llegar a AEP (se calcula en base a la distancia y velocidad)
Se registra el minuto en que aterrizó. Sirve para:
- Saber cuánto tardó en llegar (`landed_minute - minuto_aparicion`).  
- Calcular atrasos comparando con el tiempo ideal.  
- Controlar la separación mínima con el último avión que aterrizó.

Podemos calcular la pendiente de subida/bajada de velocidad a medida que avanzan en los rangos y deben bajar su velocidad 
$
m = \frac{V_{final} - V_{inicial}}{D_{final} - D_{inicial}}
$

Lo que significa que por cada milla náutica que se acerca, baja m nudos.

Ahora implementamos la simulación más básica (ver si llegó una avión al radar)

In [ ]:
#la simulación depende del umbral de lambda 
def run_simulacion(lambda_por_min):
    minutos = 1080
    aviones = []
    next_id = 1
    
    for minuto in range(minutos):
        U = random.random() #random.random() da un numero con distr uniforme entre 0 y 1
        if U < lambda_por_min: 
            nuevo = plane(id=next_id, minuto_aparicion=minuto)
            aviones.append(nuevo)
            next_id += 1
        
        #ESTO ES BASIC 
    
    return aviones


aviones1 = run_simulacion(1/60) # o sea un avión x hora

print(f"Se generaron {len(aviones1)} aviones")
print(aviones1[0].id, aviones1[0].minuto_aparicion)

def plot_aviones_por_minuto(aviones, minutos=1080):
    conteo_por_minuto = [0] * minutos
    for a in aviones:
        if 0 <= a.minuto_aparicion < minutos:
            conteo_por_minuto[a.minuto_aparicion] += 1
    plt.figure(figsize=(12, 4))
    plt.plot(range(minutos), conteo_por_minuto, drawstyle="steps-mid")
    plt.title("Aviones generados por minuto")
    plt.xlabel("Minuto")
    plt.ylabel("Cantidad de aviones")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    aviones1 = run_simulacion(1/60)  # un avión por hora en promedio
    plot_aviones_por_minuto(aviones1)
